# **Weather Load Transform Clean and extract**

## Objectives

* Investigate world weather data, clean for US and targetted measurements, save out to zip file for use in Dashboard

## Inputs

* ghcnd yearly weather files and station_list

## Outputs

* File per year for US and target elements, pivotted to have 1 row per station per day
* unified file for US for 2000-2016 limited to stations within 10km of cities from the pollution data.

## Additional Comments

* If you have any additional comments that don't fit in the previous bullets, please state them here. 



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
import pandas as pd
current_dir = os.getcwd()
print(current_dir)
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")
current_dir = os.getcwd()
print(current_dir)

h:\VScode\March Group\March_Team_Project\jupyter_notebooks
You set a new current directory
h:\VScode\March Group\March_Team_Project


# Section 1

Section 1 content

In [21]:
# load csv files from Not_to_be_shared_to_repo folder
# apply the following column names to the file, Station_ID, Date, DataValue, MFlag, QFlag, SFlag, ObsTime
weather_df = pd.read_csv('Not_to_be_shared_to_repo/2000.csv.gz', header=None)
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34973736 entries, 0 to 34973735
Data columns (total 8 columns):
 #   Column  Dtype  
---  ------  -----  
 0   0       object 
 1   1       int64  
 2   2       object 
 3   3       int64  
 4   4       object 
 5   5       object 
 6   6       object 
 7   7       float64
dtypes: float64(1), int64(2), object(5)
memory usage: 2.1+ GB


In [22]:
# rowcount
print("Row count: ", len(weather_df))
# set the column names
weather_df.columns = ["Station_ID", "Date", "Element","DataValue", "MFlag", "QFlag", "SFlag", "ObsTime"]
# filter to only include rows where Station_ID begins with 'US'
weather_df = weather_df[weather_df['Station_ID'].str.startswith('US')]
#rowcount
print("Row count: ", len(weather_df))
# display the first 5 rows
weather_df.head()

Row count:  34973736
Row count:  20092001


,Station_ID,Date,Element,DataValue,MFlag,QFlag,SFlag,ObsTime
23965,US1COMT0005,20000101,PRCP,0,NaN,NaN,N,700.0
23966,US1COMT0005,20000101,SNOW,0,NaN,NaN,N,700.0
23967,US1COMT0005,20000101,SNWD,0,NaN,NaN,N,700.0
25336,USW00094173,20000101,TMAX,-44,NaN,NaN,W,NaN
25337,USW00094173,20000101,TMIN,-117,NaN,NaN,W,NaN


YEAR/MONTH/DAY = 8 character date in YYYYMMDD format (e.g. 19860529 = May 29, 1986)
ELEMENT = 4 character indicator of element type 
DATA VALUE = 5 character data value for ELEMENT 
M-FLAG = 1 character Measurement Flag 
Q-FLAG = 1 character Quality Flag 
S-FLAG = 1 character Source Flag 
OBS-TIME = 4-character time of observation in hour-minute format (i.e. 0700 =7:00 am)

In [23]:
ElementList = [
  "PRCP",  # Precipitation (tenths of mm)

  "SNOW",  # Snowfall (mm)
  "SNWD",  # Snow depth (mm)
  "WESD",  # Water equivalent of snow on the ground (tenths of mm)
  'WESF',  # Water equivalent of snowfall (tenths of mm)
  
  "WT01",  # Fog, ice fog, or freezing fog (may include heavy fog)
  "WT02",  # Heavy fog or heaving freezing fog (not always distinguished from fog)
  "WT03",  # Thunder
  "WT04",  # Ice pellets, sleet, snow pellets, or small hail"
  "WT05",  # Hail (may include small hail)
  "WT06",  # Glaze or rime
  "WT07",  # Dust, volcanic ash, blowing dust, blowing sand, or blowing obstruction
  "WT08",  # Smoke or haze
  "WT09",  # Blowing or drifting snow
  "WT10",  # Tornado, waterspout, or funnel cloud
  'WT11',  # High or damaging winds
  "WT12",  # Blowing spray
  "WT13",  # Mist
  "WT14",  # Drizzle
  "WT15",  # Freezing drizzle
  "WT16",  # Rain (may include freezing rain, drizzle, and freezing drizzle)  
  "WT17",  # Freezing rain
  "WT18",  # Snow, snow pellets, snow grains, or ice crystals
  "WT19",  # Unknown source of precipitation

  "WT21",  # Ground fog
  'WT22',  # Ice fog or freezing fog

  "WV01",  # Fog or ice fog at a distance
  "WV03",  # Thunderstorm in the vicinity
  "WV07",  # Duststorm or sandstorm
  "WV18",  # Snow shower or snow flurry
  "WV20",  # Rain shower (may include drizzle, freezing drizzle, and freezing rain)
  
  "TMAX",  # Maximum temperature (tenths of degrees C)
  "TMIN",  # Minimum temperature (tenths of degrees C)
  "TAVG",  # Average temperature (tenths of degrees C)

  "TSUN",  # Daily total sunshine (minutes)
  "PSUN",  # Daily total sunshine (percent of possible)

  "ACSH",  # Average cloudiness sunrise to sunset from manual observations
  "ACSC",  # Average cloudiness sunrise to sunset from 30-second ceilometer data
  "ACMC",  # Average cloudiness midnight to midnight from manual observations
  "ACMH",  # Average cloudiness midnight to midnight from 30-second ceilometer data

  "AWND",  # Average daily wind speed (tenths of meters per second)
  "PGTM",  # Peak gust time (hours and minutes, i.e., HHMM)
  ]
# row count
print(weather_df.shape[0])
# filter elements from the weather_df
weather_df = weather_df[weather_df['Element'].isin(ElementList)]
# row count
print(weather_df.shape[0])

20092001
16270520


In [24]:
# record count
print(weather_df.count())
# drop MFlag, QFlag, SFlag
weather_df = weather_df.drop(columns=['MFlag', 'QFlag', 'SFlag'])
# record count
print(weather_df.count())
# count records where DataValue = 9999
print(weather_df[weather_df.DataValue == 9999].count())
# drop rows where DataValue = 9999
weather_df_filtered = weather_df[weather_df.DataValue != 9999].copy()
# rename weather_df to weather_original
weather_original = weather_df
# rename the filtered weather_df
weather_df = weather_df_filtered
# record count
print(weather_df.count())

Station_ID    16270520
Date          16270520
Element       16270520
DataValue     16270520
MFlag          5824003
QFlag            82013
SFlag         16270520
ObsTime        6810050
dtype: int64
Station_ID    16270520
Date          16270520
Element       16270520
DataValue     16270520
ObsTime        6810050
dtype: int64
Station_ID    0
Date          0
Element       0
DataValue     0
ObsTime       0
dtype: int64
Station_ID    16270520
Date          16270520
Element       16270520
DataValue     16270520
ObsTime        6810050
dtype: int64


In [25]:
# record count
weather_df.count()
#filter out all records where DataValue = 9999.9
weather_df = weather_df[weather_df.DataValue != 9999]
# record count
weather_df.count()
# produce summary of the data by day and station, provide a count of reported data by element, ignore SFLAG, MFLAG, QFLAG or ObsTime - they are not important for this analysis
weather_df.groupby(['Station_ID', 'Date', 'Element']).count()
#pivot the data so that each element is a column, each row is a unique station and date, and the value is the DataValue
weather_df = weather_df.pivot_table(index=['Station_ID', 'Date'], columns='Element', values='DataValue')
weather_df.head()




,Element,ACMH,ACSH,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,...,WT17,WT18,WT19,WT21,WT22,WV01,WV03,WV07,WV18,WV20
Station_ID,Date,,,,,,,,,,,,,,,,,,,,,
US1ALAT0002,20000703,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
US1ALLD0029,20000729,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
US1ARBT0034,20000709,NaN,NaN,NaN,NaN,69.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
US1ARLK0001,20000416,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
US1AZMR0170,20000203,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Load list of Stations from ghcnd=stations.txt, apply column headings and drop unrequired columns then view data

In [26]:
# load the ghcnd-stations.txt file - fixed width file no header
stations_df = pd.read_fwf('Source_Data/ghcnd-stations.txt', header=None)
stations_df.columns = [ "StationId","Latitude", "Longitude", "Elevation", "Name", "GSN_Flag", "HCN_CRN_Flag", "WMO_ID"]
# Drop GSN_Flag, HCN_CRN_Flag, WMO_ID columns
stations_df.drop(["GSN_Flag", "HCN_CRN_Flag", "WMO_ID"], axis=1, inplace=True)
# filter to only include rows where Station_ID begins with 'US'
stations_df = stations_df[stations_df['StationId'].str.startswith('US')]
# record count
stations_df

,StationId,Latitude,Longitude,Elevation,Name
52997,US009052008,43.7333,-96.6333,482.0,SIOUX FALLS (ENVIRON. CAN
52998,US10RMHS145,40.5268,105.1113,1569.1,RMHS 1.6 SSW
52999,US10adam001,40.5680,-98.5069,598.0,JUNIATA 1.5 S
53000,US10adam002,40.5093,-98.5493,601.1,JUNIATA 6.0 SSW
53001,US10adam003,40.4663,-98.6537,615.1,HOLSTEIN 0.1 NW
...,...,...,...,...,...
128839,USW00096405,60.4731,145.3542,25.3,CORDOVA 14 ESE
128840,USW00096406,64.5014,154.1297,78.9,RUBY 44 ESE
128841,USW00096407,66.5620,159.0036,6.7,SELAWIK 28 E
128842,USW00096408,63.4519,150.8747,678.2,DENALI 27 N


In [ ]:
# Sort stations_df by Latitude, Longitude, Name
stations_df.sort_values(['Latitude', 'Longitude', 'Name'], inplace=True)
stations_df

Load countries data, apply column names, then load states data, apply column names

In [12]:
# load ghcnd-countries.txt file - fixed width file no header
countries_df = pd.read_fwf('Source_Data/ghcnd-countries.txt', header=None)
countries_df.columns = ["CountryCode", "Name"]

# Load ghcnd-states.txt file - fixed width file no header
states_df = pd.read_fwf('Source_Data/ghcnd-states.txt', header=None)
states_df.columns = ["StateCode", "Name"]

print(countries_df.head())
print(states_df.head())

  CountryCode                  Name
0          AC   Antigua and Barbuda
1          AE  United Arab Emirates
2          AF           Afghanistan
3          AG               Algeria
4          AJ            Azerbaijan
  StateCode            Name
0        AB         ALBERTA
1        AK          ALASKA
2        AL         ALABAMA
3        AR        ARKANSAS
4        AS  AMERICAN SAMOA


---

Remove country id from beginning of station_id - as explained in GHCND_readme.txt, then drop unrequired columns from data

In [13]:
# add column to weather_df for the country code = set it to the first 2 characters of the Station_ID
weather_df['CountryCode'] = weather_df['Station_ID'].str[:2]
weather_df.head()
# drop weather_df MFlag, QFlag, SFlag, ObsTime columns
# weather_df.drop(["MFlag", "QFlag", "SFlag", "ObsTime"], axis=1, inplace=True)

,Station_ID,Date,Element,DataValue,CountryCode
0,AE000041196,20000101,TMAX,278,AE
1,AE000041196,20000101,TMIN,121,AE
2,AE000041196,20000101,TAVG,186,AE
3,AEM00041194,20000101,TMAX,251,AE
4,AEM00041194,20000101,TMIN,135,AE


Calculate records by Country and deduplicate the Weather data

In [14]:
# count of rows in weather_df by country
print(weather_df['CountryCode'].value_counts())
# deduplicate the weather_df by station_id, element, date, taking the first record if multiple records exist
weather_df = weather_df.drop_duplicates(subset=["Station_ID", "Element", "Date"], keep='first')
# count of rows in weather_df by country
print(weather_df['CountryCode'].value_counts())

CountryCode
US    20092001
CA     2982729
AS     2916911
MX     2783511
RS      936333
        ...   
SX         188
RW         115
ZA          48
EK           3
IO           1
Name: count, Length: 196, dtype: int64
CountryCode
US    20092001
CA     2982729
AS     2916911
MX     2783511
RS      936333
        ...   
SX         188
RW         115
ZA          48
EK           3
IO           1
Name: count, Length: 196, dtype: int64


As no duplications identified = this step can be ignored in the future.

From stationid extract the country code and count records

In [15]:
# seperate the countryid out of the stations_df based on 1st 2 chracters of the StationId
stations_df['CountryCode'] = stations_df['StationId'].str[:2]
# count stations by country
print(stations_df['CountryCode'].value_counts())

CountryCode
US    75846
AS    17088
CA     9269
BR     5989
MX     5249
      ...  
RW        1
SB        1
SE        1
CB        1
NN        1
Name: count, Length: 219, dtype: int64


Confirm that there are no rogue country codes in the stations_df, then filter both files for data from US only , then produce list of Elements in weather data for US records

In [16]:
# check that country code is only country code
# count of rows where CountryCode in stations_df is not in countries_df
print(stations_df[~stations_df['CountryCode'].isin(countries_df['CountryCode'])].shape[0])
# count of rows in weather_df where CountryCode is not in countries_df
print(weather_df[~weather_df['CountryCode'].isin(countries_df['CountryCode'])].shape[0])
# count rows in stations_df
print(stations_df.shape[0])
# count rows in weather_df
print(weather_df.shape[0])
# filter stations_df to only include rows where CountryCode ="US"
stations_df = stations_df[stations_df['CountryCode']=="US"]
# filter weather_df to only include rows where CountryCode ="US"
weather_df = weather_df[weather_df['CountryCode']=="US"]
# count rows in stations_df
print(stations_df.shape[0])
# count rows in weather_df
print(weather_df.shape[0])
# count of Elements in weather_df
print(weather_df['Element'].value_counts())
# list of elements with at data in at least 90% of dates and stations
elements = weather_df['Element'].value_counts()
elements = elements[elements > 0.9 * 365 * 10]
elements = elements.index.tolist()
print(elements)


0
0
129657
34973736
75846
20092001
Element
PRCP    3227238
SNOW    2799342
SNWD    2763856
TMAX    2658783
TMIN    2657783
         ...   
MDSF         95
DASF         87
WV07         74
WT10         72
WV18          4
Name: count, Length: 87, dtype: int64
['PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'TOBS', 'TAVG', 'WESD', 'AWND', 'WSF5', 'WDF2', 'WSF2', 'WDF5', 'FMTM', 'PGTM', 'WT01', 'WT03', 'EVAP', 'WT13', 'WDMV', 'WT16', 'TSUN', 'MNPN', 'MXPN', 'SX02', 'SN02', 'WESF', 'SN32', 'SX32', 'WT08', 'WT18', 'WT02', 'SX03', 'SN03', 'SN52', 'SX52', 'SN01', 'SX01', 'WT11', 'WT04', 'WT06', 'WT05', 'MDPR', 'DAPR', 'SX12', 'SN12', 'WT22', 'SN31', 'SX31']


Filter weather for chosen elements, pivot data to produce 1 line per station per day and then perform a comparison of rows containing all elements and those only with the chosen elements

In [ ]:
ElementList = [
  "PRCP",  # Precipitation (tenths of mm)



  "SNOW",  # Snowfall (mm)
  "SNWD",  # Snow depth (mm)
  "WESD",  # Water equivalent of snow on the ground (tenths of mm)
  'WESF',  # Water equivalent of snowfall (tenths of mm)
  "WT01",  # Fog, ice fog, or freezing fog (may include heavy fog)
  "WT02",  # Heavy fog or heaving freezing fog (not always distinguished from fog)
  "WT03",  # Thunder
  "WT04",  # Ice pellets, sleet, snow pellets, or small hail"
  "WT05",  # Hail (may include small hail)
  "WT06",  # Glaze or rime
  "WT07",  # Dust, volcanic ash, blowing dust, blowing sand, or blowing obstruction
  "WT08",  # Smoke or haze
  "WT09",  # Blowing or drifting snow
  "WT10",  # Tornado, waterspout, or funnel cloud
  'WT11',  # High or damaging winds
  "WT12",  # Blowing spray
  "WT13",  # Mist
  "WT14",  # Drizzle
  "WT15",  # Freezing drizzle
  "WT16",  # Rain (may include freezing rain, drizzle, and freezing drizzle)  
  "WT17",  # Freezing rain
  "WT18",  # Snow, snow pellets, snow grains, or ice crystals
  "WT19",  # Unknown source of precipitation

  "WT21",  # Ground fog
  'WT22',  # Ice fog or freezing fog

  "WV01",  # Fog or ice fog at a distance
  "WV03",  # Thunderstorm in the vicinity
  "WV07",  # Duststorm or sandstorm
  "WV18",  # Snow shower or snow flurry
  "WV20",  # Rain shower (may include drizzle, freezing drizzle, and freezing rain)
  
  "TMAX",  # Maximum temperature (tenths of degrees C)
  "TMIN",  # Minimum temperature (tenths of degrees C)
  "TAVG",  # Average temperature (tenths of degrees C)

  "TSUN",  # Daily total sunshine (minutes)
  "PSUN",  # Daily total sunshine (percent of possible)

  "ACSH",  # Average cloudiness sunrise to sunset from manual observations
  "ACSC",  # Average cloudiness sunrise to sunset from 30-second ceilometer data
  "ACMC",  # Average cloudiness midnight to midnight from manual observations
  "ACMH",  # Average cloudiness midnight to midnight from 30-second ceilometer data

  "AWND",  # Average daily wind speed (tenths of meters per second)
  "PGTM",  # Peak gust time (hours and minutes, i.e., HHMM)
  ]

# save Stations_df to a csv file called Us_Stations.csv
stations_df.to_csv("Us_Stations.csv", index=False)
# filter weather_df to only include rows where Element in ElementList
weather_df = weather_df[weather_df['Element'].isin(ElementList)]
# count rows in weather_df
print(weather_df.shape[0])
# unpack weather_df Element in to seperate columns with value eq to DataValue where Element = TMAX, grouped by Station_Id and Date
weather_up_df = weather_df.pivot_table(index=["Station_ID", "Date"],
                     columns="Element",
                     values="DataValue",
                     aggfunc='first').reset_index()

# Ensure a column exists for each value in ElementList
for element in ElementList:
  if element not in weather_up_df.columns:
    weather_up_df[element] = None
# count rows of weather_df
print(weather_up_df.shape[0])
# save weather_df to a csv file called Us_Weather_Unpacked.zip
weather_up_df.to_csv("Us_Weather_Unpacked.zip", index=False, compression='zip')

16255417
3604040


Quick look at a single station to confirm that the pivot_table is working correctly

In [ ]:
weather_up_df[weather_up_df["Station_ID"] == "USW00094173"].head(100)

Do the same for a single element in the original data format

In [ ]:
# weather_df.head(100)
weather_df[weather_df["Element"] == "WT01"].head(100)

# Scripts to bulk load and clean yearly data files using techniks from above

Load annual data, filter for US and required elements, drop unrequired columns, perform pivot and resave as a zip file, print file name created and row/column counts

In [30]:
ElementList = [
  "PRCP",  # Precipitation (tenths of mm)



  "SNOW",  # Snowfall (mm)
  "SNWD",  # Snow depth (mm)
  "WESD",  # Water equivalent of snow on the ground (tenths of mm)
  'WESF',  # Water equivalent of snowfall (tenths of mm)
  "WT01",  # Fog, ice fog, or freezing fog (may include heavy fog)
  "WT02",  # Heavy fog or heaving freezing fog (not always distinguished from fog)
  "WT03",  # Thunder
  "WT04",  # Ice pellets, sleet, snow pellets, or small hail"
  "WT05",  # Hail (may include small hail)
  "WT06",  # Glaze or rime
  "WT07",  # Dust, volcanic ash, blowing dust, blowing sand, or blowing obstruction
  "WT08",  # Smoke or haze
  "WT09",  # Blowing or drifting snow
  "WT10",  # Tornado, waterspout, or funnel cloud
  'WT11',  # High or damaging winds
  "WT12",  # Blowing spray
  "WT13",  # Mist
  "WT14",  # Drizzle
  "WT15",  # Freezing drizzle
  "WT16",  # Rain (may include freezing rain, drizzle, and freezing drizzle)  
  "WT17",  # Freezing rain
  "WT18",  # Snow, snow pellets, snow grains, or ice crystals
  "WT19",  # Unknown source of precipitation

  "WT21",  # Ground fog
  'WT22',  # Ice fog or freezing fog

  "WV01",  # Fog or ice fog at a distance
  "WV03",  # Thunderstorm in the vicinity
  "WV07",  # Duststorm or sandstorm
  "WV18",  # Snow shower or snow flurry
  "WV20",  # Rain shower (may include drizzle, freezing drizzle, and freezing rain)
  
  "TMAX",  # Maximum temperature (tenths of degrees C)
  "TMIN",  # Minimum temperature (tenths of degrees C)
  "TAVG",  # Average temperature (tenths of degrees C)

  "TSUN",  # Daily total sunshine (minutes)
  "PSUN",  # Daily total sunshine (percent of possible)

  "ACSH",  # Average cloudiness sunrise to sunset from manual observations
  "ACSC",  # Average cloudiness sunrise to sunset from 30-second ceilometer data
  "ACMC",  # Average cloudiness midnight to midnight from manual observations
  "ACMH",  # Average cloudiness midnight to midnight from 30-second ceilometer data

  "AWND",  # Average daily wind speed (tenths of meters per second)
  "PGTM",  # Peak gust time (hours and minutes, i.e., HHMM)
  ]

weather_df  = pd.DataFrame()
weather_up_df = pd.DataFrame()

# list of files to load
FileToLoad = [  # "2000.csv.gz",
              # "2001.csv.gz",
              # "2002.csv.gz",
              "2003.csv.gz",
              # "2004.csv.gz",
              # "2005.csv.gz",
              # "2006.csv.gz",
              # "2007.csv.gz",
              # "2008.csv.gz",
              # "2009.csv.gz",
              # "2010.csv.gz",
              # "2011.csv.gz",
              # "2012.csv.gz",
              # "2013.csv.gz",
              # "2014.csv.gz",
              # "2015.csv.gz",
              # "2016.csv.gz",
              ]
# load weather stations within 10Km of pollution data cities
stations_df = pd.read_csv("Outputs\\Us_Stations_with_City_100km.csv")

# load a file from the list above then process it and append it to weather_df
for file in FileToLoad:
    # add filename to path "Not_to_be_shared_to_repo/"
    filepath = os.path.join("Not_to_be_shared_to_repo", file)
    # setup df
    weather_df  = pd.DataFrame()
    weather_up_df = pd.DataFrame()
    # Print filepath
    print(filepath)
    # load the file
    weather_df = pd.read_csv(filepath, header=None)
    # Name columns
    weather_df.columns = ["Station_ID", "Date", "Element", "DataValue", "MFlag", "QFlag", "SFlag", "ObsTime"]
    # print the shape of weather_df
    print("Pre filter:", weather_df.shape)
    # Extract Country Code
    weather_df['CountryCode'] = weather_df['Station_ID'].str[:2]
    # Drop unneccesary columns
    weather_df.drop(["MFlag", "QFlag", "SFlag", "ObsTime"], axis=1, inplace=True)
    # Deduplicate weather_df
    weather_df = weather_df.drop_duplicates(subset=["Station_ID", "Element", "Date"], keep='first')
    # filter weather_df to only include rows where CountryCode ="US"
    weather_df = weather_df[
        (weather_df['CountryCode'] == "US") &
        (weather_df['Station_ID'].isin(stations_df['StationId'])) &
        (weather_df['Element'].isin(ElementList)) &
        (weather_df['DataValue'] != 9999)
    ]
    # print the shape of weather_df
    print("Post Filter:", weather_df.shape)
    # filter out all records where DataValue = 9999.9
    # Define rules
    def get_agg_type(element):
        if element == "TMAX":
            return "max"
        elif element == "TMIN":
            return "min"
        elif element.startswith("WT") or element.startswith("WV"):
            return "max"
        else:
            return "mean"

    # Add aggregation type
    weather_df["AggType"] = weather_df["Element"].map(get_agg_type)

    # Use groupby and aggregation
    def custom_group(df):
        agg_type = df["AggType"].iloc[0]
        if agg_type == "max":
            return df["DataValue"].max()
        elif agg_type == "min":
            return df["DataValue"].min()
        else:
            return df["DataValue"].mean()

    weather_df = (
        weather_df
        .groupby(["Station_ID", "Date", "Element"])
        .apply(custom_group)
        .reset_index(name="DataValue")
    )
    
    # count rows in weather_df
    print(weather_df.shape[0])
    # perform check that there is only 1 value for each Station_ID, Date, Element
    print(weather_df.groupby(['Station_ID', 'Date', 'Element']).count().max())
    # print the shape of weather_df
    print(weather_df.shape)
    # add filename to path "Not_to_be_shared_to_repo/"
    filename = f"Us_{file}_Weather_Unpacked.zip"
    filepath = os.path.join("Not_to_be_shared_to_repo", filename)
    # implement save append to csv file
    weather_df.to_csv(filepath, index=False, header=True, compression='zip')
    # print file name
    print(filename)

weather_df

Not_to_be_shared_to_repo\2003.csv.gz
Pre filter: (36590147, 8)
Post Filter: (3834501, 5)
3834501
DataValue    1
dtype: int64
(3834501, 4)
Us_2003.csv.gz_Weather_Unpacked.zip


,Station_ID,Date,Element,DataValue
0,US1ALJF0025,20030506,PRCP,0.0
1,US1ALJF0025,20030506,SNOW,0.0
2,US1ALJF0025,20030507,PRCP,86.0
3,US1FLBV0002,20030101,PRCP,203.0
4,US1FLBV0002,20030101,SNOW,0.0
...,...,...,...,...
3834496,USW00094996,20031230,TMAX,91.0
3834497,USW00094996,20031230,TMIN,-33.0
3834498,USW00094996,20031231,PRCP,0.0
3834499,USW00094996,20031231,TMAX,33.0


Load pivotted, filtered data by year and filter for required cities - append into single file and save.

In [ ]:

FileToLoad = [
    "Us_2000.csv.gz_Weather_Unpacked.zip",
    "Us_2001.csv.gz_Weather_Unpacked.zip",
    "Us_2002.csv.gz_Weather_Unpacked.zip",
    "Us_2003.csv.gz_Weather_Unpacked.zip",
    "Us_2004.csv.gz_Weather_Unpacked.zip",
    "Us_2005.csv.gz_Weather_Unpacked.zip",
    "Us_2006.csv.gz_Weather_Unpacked.zip",
    "Us_2007.csv.gz_Weather_Unpacked.zip",
    "Us_2008.csv.gz_Weather_Unpacked.zip",
    "Us_2009.csv.gz_Weather_Unpacked.zip",
    "Us_2010.csv.gz_Weather_Unpacked.zip",
    "Us_2011.csv.gz_Weather_Unpacked.zip",
    "Us_2012.csv.gz_Weather_Unpacked.zip",
    "Us_2013.csv.gz_Weather_Unpacked.zip",
    "Us_2014.csv.gz_Weather_Unpacked.zip",
    "Us_2015.csv.gz_Weather_Unpacked.zip",
    "Us_2016.csv.gz_Weather_Unpacked.zip",
    ]

weather_df = pd.DataFrame()
weather_up_df = pd.DataFrame()

# load stations_df from "Outputs//Us_Stations_with_City_100km.csv"
stations_df = pd.read_csv("Outputs//Us_Stations_with_City_100km.csv")

# load file from list and append to weather_final_df
for file in FileToLoad:
    # set path for fileload
    loadfile = "Not_to_be_shared_to_repo/"
    filepath = os.path.join(loadfile, file)
    # load file
    weather_df = pd.read_csv(filepath)
    # filter to only keep stations that are in stations_df
    weather_df = weather_df[weather_df['Station_ID'].isin(stations_df['StationId'])]
    # append to weather_final_df
    weather_up_df = pd.concat([weather_up_df, weather_df], ignore_index=True)
    # print file name
    print(file)
    # print shape of weather_final_df
    print(weather_up_df.shape)
    # drop weather_df

# save weather_final_df to a csv file called Us_Weather_Final.csv
weather_up_df.to_csv("Not_to_be_shared_to_repo/Us_Weather_Final_10km_V2.zip", index=True, header=True, compression='zip')

Us_2000.csv.gz_Weather_Unpacked.zip
(806673, 37)
Us_2001.csv.gz_Weather_Unpacked.zip
(1611936, 37)
Us_2002.csv.gz_Weather_Unpacked.zip
(2424019, 37)
Us_2003.csv.gz_Weather_Unpacked.zip
(3253703, 37)
Us_2004.csv.gz_Weather_Unpacked.zip
(4101920, 37)
Us_2005.csv.gz_Weather_Unpacked.zip
(4960286, 37)
Us_2006.csv.gz_Weather_Unpacked.zip
(5952474, 38)
Us_2007.csv.gz_Weather_Unpacked.zip
(7166973, 38)
Us_2008.csv.gz_Weather_Unpacked.zip
(8704104, 38)
Us_2009.csv.gz_Weather_Unpacked.zip
(10436932, 38)
Us_2010.csv.gz_Weather_Unpacked.zip
(12244417, 38)
Us_2011.csv.gz_Weather_Unpacked.zip
(14097391, 38)
Us_2012.csv.gz_Weather_Unpacked.zip
(16060186, 38)
Us_2013.csv.gz_Weather_Unpacked.zip
(18062137, 38)
Us_2014.csv.gz_Weather_Unpacked.zip
(20032007, 38)
Us_2015.csv.gz_Weather_Unpacked.zip
(22013543, 38)
Us_2016.csv.gz_Weather_Unpacked.zip
(24010324, 38)


---

In [28]:
import pandas as pd

# import csv from source/simplemaps-worldcities-basic.csv
cities_df = pd.read_csv("Source_Data\\simplemaps_uscities_basicv1.90.zip")
# import csv pollution_data_available.csv
pollution_df = pd.read_csv("Outputs\\pollution_data_available.csv")
# update pollution_df with lat and long from cities_df
pollution_df = pollution_df.merge(cities_df, left_on='City', right_on='city', how='left')
#show data
pollution_df
# save pollution_df to a csv file called city_data_inc_latlog_.csv
pollution_df.to_csv("city_data_inc_latlog_.csv", index=False)


Augment the Pollution Dataset with Coordinates:

Retrieve the latitude and longitude for each city in the pollution dataset. Resources like the SimpleMaps US Cities Database provide comprehensive data on U.S. cities, including their geographic coordinates.

In [29]:
# Load the city_data_inc_latlog_.csv file
city_df = pd.read_csv("city_data_inc_latlog_.csv")
#drop unneccesary columns
# city_df.drop(["city", "admin_name", "population", "id", "zips", "County", "State", "incorporated", "source", "ranking", "timezone"], axis=1, inplace=True)
#show data
city_df

,City,County,State,Min Year,Max Year,Year Difference,city,city_ascii,state_id,state_name,...,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,Vallejo,Solano,California,2000,2015,15,Vallejo,Vallejo,CA,California,...,-122.2342,171414.0,1581.2,shape,False,True,America/Los_Angeles,2.0,94592 94591 94589 94590,1.840021e+09
1,Phoenix,Maricopa,Arizona,2000,2015,15,Phoenix,Phoenix,AZ,Arizona,...,-112.0892,4065338.0,1210.3,shape,False,True,America/Phoenix,1.0,85009 85003 85006 85007 85004 85083 85086 8508...,1.840021e+09
2,Phoenix,Maricopa,Arizona,2000,2015,15,Phoenix,Phoenix,OR,Oregon,...,-122.8154,4383.0,1184.7,shape,False,True,America/Los_Angeles,3.0,97535,1.840020e+09
3,Phoenix,Maricopa,Arizona,2000,2015,15,Phoenix,Phoenix,NY,New York,...,-76.2961,2393.0,789.3,shape,False,True,America/New_York,3.0,13135,1.840004e+09
4,Phoenix,Maricopa,Arizona,2000,2015,15,Phoenix,Phoenix,IL,Illinois,...,-87.6308,1284.0,1069.3,shape,False,True,America/Chicago,3.0,60426,1.840011e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,Boston,Suffolk,Massachusetts,2003,2013,10,Boston,Boston,GA,Georgia,...,-83.7899,1305.0,223.6,shape,False,True,America/New_York,3.0,31626,1.840014e+09
93,Boston,Suffolk,Massachusetts,2003,2013,10,Boston,Boston,IN,Indiana,...,-84.8517,262.0,89.5,shape,False,True,America/Indiana/Indianapolis,3.0,47324,1.840009e+09
94,Boston,Suffolk,Massachusetts,2003,2013,10,Boston,Boston,VA,Virginia,...,-75.8437,156.0,23.5,shape,False,False,America/New_York,3.0,23420,1.840025e+09
95,Boston,Suffolk,Massachusetts,2003,2013,10,Boston,Boston,KY,Kentucky,...,-85.6804,154.0,30.3,shape,False,False,America/New_York,3.0,40107,1.840027e+09


Calculate Distances Between Cities and Stations:

Utilize the Haversine formula to compute the great-circle distance between each city and all monitoring stations. This formula calculates the shortest distance over the Earth's surface, providing an accurate measure between two points specified by their latitude and longitude.

In [30]:
import numpy as np
# load US_Stations.csv
stations_df = pd.read_csv("Source_Data\\Us_Stations.csv")
#

# Function to calculate Haversine distance
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    return R * c  # Distance in km

# using the lat and long from the stations_df and city_df calculate the distance between the two
stations_df['City'] = None
stations_df['CityDistance'] = None
for i, station in stations_df.iterrows():
    lat1, lon1 = station['Latitude'], station['Longitude']
    min_distance = np.inf
    for j, city in city_df.iterrows():
        lat2, lon2 = city['lat'], city['lng']
        distance = haversine(lat1, lon1, lat2, lon2)
        if distance < min_distance:
            min_distance = distance
            stations_df.at[i, 'City'] = city['city']
            stations_df.at[i, 'CityDistance'] = min_distance

# display data
stations_df

,StationId,Latitude,Longitude,Elevation,Name,CountryCode,City,CityDistance
0,US009052008,43.7333,-96.6333,482.0,SIOUX FALLS (ENVIRON. CAN,US,Concord,152.803597
1,US10RMHS145,40.5268,105.1113,1569.1,RMHS 1.6 SSW,US,Long Beach,9194.911455
2,US10adam001,40.5680,-98.5069,598.0,JUNIATA 1.5 S,US,Washington,149.439968
3,US10adam002,40.5093,-98.5493,601.1,JUNIATA 6.0 SSW,US,Washington,149.005169
4,US10adam003,40.4663,-98.6537,615.1,HOLSTEIN 0.1 NW,US,Washington,154.405664
...,...,...,...,...,...,...,...,...
75841,USW00096405,60.4731,145.3542,25.3,CORDOVA 14 ESE,US,Long Beach,5696.866306
75842,USW00096406,64.5014,154.1297,78.9,RUBY 44 ESE,US,Long Beach,5107.900915
75843,USW00096407,66.5620,159.0036,6.7,SELAWIK 28 E,US,Long Beach,4831.525021
75844,USW00096408,63.4519,150.8747,678.2,DENALI 27 N,US,Long Beach,5296.779038


In [22]:
# save stations_df to a csv file called Us_Stations_with_City.csv
stations_df.to_csv("Source_Data\\Us_Stations_with_City.csv", index=False)


In [23]:
# filter stations_df to provide only the stations that are within 100km of a city
stations_df = stations_df[stations_df['CityDistance'] < 100]
# display data
stations_df

,StationId,Latitude,Longitude,Elevation,Name,CountryCode,City,CityDistance
82,US10boyd002,42.9095,-98.8492,552.9,BUTTE 0.1 S,US,Dallas,65.421913
83,US10boyd003,42.9141,-98.6603,513.0,SPENCER 3.5 NE,US,Dallas,78.381858
84,US10boyd005,42.9096,-98.8428,555.0,BUTTE 0.4 ESE,US,Dallas,65.847671
85,US10brow001,42.5359,-99.7092,791.9,LONG PINE 0.4 W,US,Dallas,79.60591
88,US10brow005,42.6748,-99.7695,686.1,LONG PINE 15.7 ESE,US,Dallas,65.886702
...,...,...,...,...,...,...,...,...
75828,USW00094978,41.7631,-96.1797,312.1,TEKAMAH MUNI AP,US,Washington,40.707
75829,USW00094982,41.6117,-90.5808,259.4,DAVENPORT,US,Washington,99.014447
75833,USW00094990,43.3892,-99.8433,615.1,WINNER WILEY FLD,US,Dallas,31.270658
75838,USW00094995,40.8483,-96.5650,362.4,LINCOLN 8 ENE,US,Washington,68.028597


In [ ]:
# save stations_df to a csv file called Us_Stations_with_City_100km.csv
stations_df.to_csv("Source_Data\\Us_Stations_with_City_100km.csv", index=False)